In [ ]:
pip install -r requirements.txt

In [19]:
import sys
import os

# Add root directory or other specific directories
sys.path.append('/Users/minzhu/Desktop/DLH/DLH_2023_keyclass/KeyClass')
sys.path.append('/Users/minzhu/Desktop/DLH/DLH_2023_keyclass/KeyClass/scripts')

from scripts import label_data
from keyclass import models, helper

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/minzhu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/minzhu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# Import statements
import sys
sys.path.append('../keyclass/')
sys.path.append('../scripts/')
import keyclass.helper as helper
import keyclass.models as models
import argparse
import label_data, encode_datasets, train_downstream_model
import torch
import pickle
import numpy as np
import os
from os.path import join, exists
from datetime import datetime
import create_lfs
import train_classifier
from transformers import AutoTokenizer, AutoModel

In [23]:
config_file_path = r'../config_files/default_config.yml' # Specify path to the configuration file
random_seed = 0 # Random seed for experiments

In [24]:
args = helper.Parser(config_file_path=config_file_path).parse()

if args['use_custom_encoder']:
    model = models.CustomEncoder(pretrained_model_name_or_path=args['base_encoder'], 
        device='cuda' if torch.cuda.is_available() else 'cpu')
else:
    model = models.Encoder(model_name=args['base_encoder'], 
        device='cuda' if torch.cuda.is_available() else 'cpu')


for split in ['train', 'test']:
    sentences = helper.fetch_data(dataset=args['dataset'], split=split, path=args['data_path'])
    embeddings = model.encode(sentences=sentences, batch_size=args['end_model_batch_size'], 
                                show_progress_bar=args['show_progress_bar'], 
                                normalize_embeddings=args['normalize_embeddings'])
    with open(join(args['data_path'], args['dataset'], f'{split}_embeddings.pkl'), 'wb') as f:
        pickle.dump(embeddings, f)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-mpnet-base-v2


ValueError: File train.txt does not exists in /zfsauton/project/public/chufang/classes/imdb